In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from povms.library.pm_sim_implementation import ClassicalShadows, RandomizedPMs
from povms.post_processor.povm_post_processor import POVMPostprocessor
from povms.sampler.povm_sampler import POVMSampler
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import (
    Operator,
    SparsePauliOp,
    Statevector,
    random_hermitian,
)
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session

# Random Circuit

We first llok at a random the 2-qubit circuit, with depth 3.

In [3]:
n_qubit = 3
n_cicuits = 5
np.random.seed(10)
seeds= np.random.randint(0,10000000,size=n_cicuits)
qc_random = [random_circuit(num_qubits=n_qubit, depth = 3, measure=False, seed=seed) for seed in seeds]
qc_random[0].draw()

┌──────┐┌──────┐                                                
q_0: ─────┤1     ├┤0     ├──────────────────────────────■─────────────────
     ┌───┐│      ││      │    ┌───┐    ┌────────────────┴────────────────┐
q_1: ┤ H ├┤  Dcx ├┤  Ecr ├────┤ S ├────┤ U(3.4168,2.3054,0.68636,1.6934) ├
     └───┘│      ││      │┌───┴───┴───┐└─────────────────────────────────┘
q_2: ─────┤0     ├┤1     ├┤ P(5.0287) ├───────────────────────────────────
          └──────┘└──────┘└───────────┘

We also draw some random observables 

In [4]:
set_observables = [SparsePauliOp.from_operator(random_hermitian(2**n_qubit)) for _ in range(10)]

For reference, we compute the true final state and the exact expectation values for the different observables.

In [5]:
exp_val = np.zeros((len(qc_random), len(set_observables)), dtype=complex)
for i, qc in enumerate(qc_random):
    exact_state = Statevector(np.array(Operator(qc).data[:,0]).squeeze())
    exp_val[i] = np.array([exact_state.expectation_value(obs) for obs in set_observables])
exp_val = np.real_if_close(exp_val)

# Classical Shadows

We now look at the implementation of Classical Shaodws measurement

In [6]:
# By default, the Classical Shadows (CS) measurement uses X,Y,Z measurements with equal probability.
cs_implementation = ClassicalShadows(n_qubit=n_qubit)
# Define the default shot budget.
cs_shots = 4096

In [7]:
pubs = []
for qcr in qc_random:
    pubs.append((qcr, None, np.random.randint(100000,1000000) if np.random.randint(2)==0 else None, cs_implementation))
print(pubs)

[(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1757a58a0>, None, None, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x1757a7fa0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1757a6fb0>, None, 399741, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x1757a7fa0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1757a7dc0>, None, 208445, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x1757a7fa0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1757a5db0>, None, None, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x1757a7fa0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1757a7bb0>, None, 240296, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x1757a7fa0>)]


In [8]:
# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
backend = AerSimulator()
with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    cs_job = sampler.run(pubs, shots=cs_shots)

/Users/dao/Documents/Internship/ic-povms/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/session.py:156: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


## Results
We retrieve the result object, which contains the POVM used and from which we can query the counts of each outcome.

In [9]:
cs_result = cs_job.result()
sum(cs_result[1].get_counts().values())

print(len(cs_job.book_keeping), cs_job.book_keeping)
print(len(cs_job.pvm_keys),cs_job.pvm_keys)
raw_result = cs_job.base_job.result()
print(len(raw_result))
slice_start = 0
for i, slice_length in enumerate(cs_job.book_keeping):
    pub_result = raw_result[slice_start : slice_start + slice_length]
    print(len(pub_result), pub_result)
    slice_start += slice_length
    print(cs_job.pvm_keys[i])

5 [27, 27, 27, 27, 27]
5 [[(0, 2, 2), (0, 2, 0), (2, 1, 2), (2, 1, 1), (1, 1, 0), (2, 2, 1), (0, 0, 2), (2, 1, 0), (1, 0, 2), (0, 1, 2), (1, 0, 1), (2, 0, 1), (1, 1, 2), (0, 2, 1), (1, 0, 0), (1, 2, 2), (2, 2, 0), (1, 1, 1), (2, 0, 2), (0, 1, 0), (1, 2, 1), (0, 0, 1), (0, 0, 0), (0, 1, 1), (1, 2, 0), (2, 2, 2), (2, 0, 0)], [(1, 0, 2), (2, 0, 2), (0, 1, 1), (2, 1, 2), (2, 2, 2), (1, 1, 2), (0, 2, 0), (2, 0, 0), (1, 0, 0), (1, 2, 1), (0, 2, 1), (0, 1, 0), (1, 0, 1), (0, 2, 2), (1, 1, 0), (0, 0, 2), (1, 2, 2), (2, 1, 0), (2, 1, 1), (0, 0, 0), (1, 2, 0), (2, 2, 1), (1, 1, 1), (0, 0, 1), (0, 1, 2), (2, 0, 1), (2, 2, 0)], [(2, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 2), (1, 2, 2), (2, 0, 2), (0, 0, 2), (0, 1, 2), (1, 0, 0), (1, 1, 1), (1, 2, 1), (2, 2, 2), (0, 2, 2), (0, 2, 1), (0, 1, 0), (1, 2, 0), (2, 2, 0), (1, 0, 2), (1, 1, 0), (2, 2, 1), (0, 2, 0), (2, 1, 1), (2, 1, 0), (0, 0, 1), (2, 0, 0), (2, 1, 2), (0, 0, 0)], [(2, 1, 0), (2, 1, 1), (2, 2, 2), (1, 0, 0), (1, 2, 1), (2, 2, 1), (0, 0, 2),

In [10]:
cs_result = cs_job.result()
print(f"POVM: {cs_result[0].povm}")
print(f"Counts: {cs_result[1].get_counts()}")

POVM: <povms.library.pm_sim_implementation.ClassicalShadows object at 0x1757a7fa0>
Counts: {(3, 1, 4): 1845, (2, 1, 5): 1869, (3, 0, 5): 1766, (2, 1, 4): 1914, (2, 0, 4): 1925, (3, 0, 4): 1876, (3, 1, 5): 1846, (2, 0, 5): 1855, (5, 0, 5): 1861, (4, 1, 5): 1774, (4, 0, 4): 1808, (4, 1, 4): 1802, (5, 0, 4): 1850, (5, 1, 4): 1840, (4, 0, 5): 1843, (5, 1, 5): 1849, (0, 2, 3): 3791, (0, 3, 3): 3612, (0, 3, 2): 3759, (0, 2, 2): 3724, (4, 2, 5): 1856, (4, 3, 4): 1862, (5, 3, 4): 1829, (5, 2, 5): 1854, (5, 3, 5): 1813, (4, 3, 5): 1917, (5, 2, 4): 1815, (4, 2, 4): 1843, (4, 4, 5): 3769, (5, 4, 4): 3756, (4, 4, 4): 3824, (5, 4, 5): 3747, (3, 2, 4): 1804, (2, 2, 5): 1912, (2, 3, 5): 1874, (2, 3, 4): 1781, (3, 2, 5): 1825, (2, 2, 4): 1950, (3, 3, 5): 1861, (3, 3, 4): 1913, (0, 4, 0): 14834, (5, 1, 0): 3674, (4, 0, 0): 3703, (5, 0, 0): 3637, (4, 1, 0): 3650, (2, 1, 0): 3742, (3, 1, 0): 3660, (3, 0, 0): 3717, (2, 0, 0): 3687, (3, 4, 3): 3679, (2, 4, 3): 3662, (3, 4, 2): 3664, (2, 4, 2): 3809, (0, 4,

We now define our POVM post-processor, which will use the resukt object to estimate expectation values of some observables.

In [11]:
for i_state in range(len(cs_result)):
    cs_postprocessor = POVMPostprocessor(cs_result[i_state])
    cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs) for obs in set_observables]
    cs_shots = sum(cs_postprocessor.counts.values())

    n = int(np.ceil(np.log10(cs_shots)))
    print('Exact        CS')
    print(f'             ({cs_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {cs_est_exp_val[i]:>{8+n}.3e}   {abs(cs_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        CS
             (4096 shots)
-8.507e-01     -1.122e+00      31.9%
-1.020e-01     -7.602e-02      25.5%
-3.536e-01     -4.683e-01      32.4%
 8.492e-01      9.775e-01      15.1%
 1.152e+00      1.220e+00       5.9%
-1.868e-02      5.103e-02     373.2%
-1.307e-01     -2.504e-01      91.6%
 4.855e-01      3.965e-01      18.3%
 8.988e-01      9.627e-01       7.1%
-1.147e+00     -1.244e+00       8.4%
Exact        CS
             (399741 shots)
 8.480e-01        8.268e-01     2.5%
-2.542e+00       -2.568e+00     1.1%
 7.712e-02        7.810e-02     1.3%
-8.587e-01       -8.569e-01     0.2%
-1.164e+00       -1.167e+00     0.2%
-6.345e-01       -6.221e-01     2.0%
-1.015e+00       -1.012e+00     0.4%
-6.353e-01       -6.103e-01     3.9%
 1.555e+00        1.561e+00     0.4%
-1.332e+00       -1.338e+00     0.5%
Exact        CS
             (208445 shots)
-6.979e-01       -6.709e-01     3.9%
-5.791e-02       -7.672e-02    32.5%
 1.120e+00        1.126e+00     0.5%
-3.685e-01       -3

## Multi_jobs

In [12]:
cs_shots = 4096

with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    cs_jobs = sampler.run(pubs, povm=cs_implementation, shots=cs_shots, multi_job=True)

In [13]:
for i_state, cs_job in enumerate(cs_jobs):
    cs_result = cs_job.result()[0]
    cs_postprocessor = POVMPostprocessor(cs_result)
    cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs) for obs in set_observables]
    cs_shots = sum(cs_postprocessor.counts.values())

    n = int(np.ceil(np.log10(cs_shots)))
    print('Exact        PM-sim')
    print(f'             ({cs_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {cs_est_exp_val[i]:>{8+n}.3e}   {abs(cs_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        PM-sim
             (4096 shots)
-8.507e-01     -8.294e-01       2.5%
-1.020e-01     -2.974e-01     191.4%
-3.536e-01     -4.647e-01      31.4%
 8.492e-01      7.545e-01      11.1%
 1.152e+00      9.581e-01      16.8%
-1.868e-02     -1.336e-02      28.4%
-1.307e-01     -2.343e-01      79.3%
 4.855e-01      3.949e-01      18.7%
 8.988e-01      8.121e-01       9.6%
-1.147e+00     -9.801e-01      14.6%
Exact        PM-sim
             (399741 shots)
 8.480e-01        8.458e-01     0.3%
-2.542e+00       -2.541e+00     0.0%
 7.712e-02        6.093e-02    21.0%
-8.587e-01       -8.431e-01     1.8%
-1.164e+00       -1.164e+00     0.1%
-6.345e-01       -6.233e-01     1.8%
-1.015e+00       -1.003e+00     1.2%
-6.353e-01       -6.101e-01     4.0%
 1.555e+00        1.566e+00     0.7%
-1.332e+00       -1.334e+00     0.2%
Exact        PM-sim
             (208445 shots)
-6.979e-01       -6.590e-01     5.6%
-5.791e-02       -9.199e-02    58.8%
 1.120e+00        1.099e+00     1.9%
-3.685e

# PM-simulable POVM
We now look at POVM that are simulable (through randomization) with only single-qubit projective measurements (PMs).

In [14]:
pubs_pm = []
for pub in pubs:
    pubs_pm.append((pub[0], None, pub[2], None))

# Define our projective measurements acting on each qubit.
angles = np.array([[0.0, 0.0, 0.5 * np.pi, 0.0, 0.5 * np.pi, 0.5 * np.pi],[1.2,0.0, 0.1,3.14,0.7,0.3], [0.2,0.1, 1.2,3.14,-0.7,0.3]])
# Set the distributions of the shots among the PMs.
bias = np.array([[0.34, 0.33, 0.33], [0.7,0.15,0.15], [0.34,0.33,0.33]])

# Define the PM-simulable POVM.
pmsim_implementation = RandomizedPMs(n_qubit=n_qubit, bias=bias, angles=angles)

In [15]:
pmsim_shots = 4096

with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    pmsim_job = sampler.run(pubs_pm, povm=pmsim_implementation, shots=pmsim_shots)

## Results

Retrieve the result of the sampling and use it to estiamte some expectation values via the post-processor

In [16]:
pmsim_result = pmsim_job.result()

for i_state in range(len(pmsim_result)):
    pmsim_postprocessor = POVMPostprocessor(pmsim_result[i_state])
    pmsim_est_exp_val = [pmsim_postprocessor.get_expectation_value(obs) for obs in set_observables]
    pmsim_shots = sum(pmsim_postprocessor.counts.values())

    n = int(np.ceil(np.log10(pmsim_shots)))
    print('Exact        PM-sim')
    print(f'             ({pmsim_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {pmsim_est_exp_val[i]:>{8+n}.3e}   {abs(pmsim_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        PM-sim
             (4096 shots)
-8.507e-01     -8.990e+00     956.8%
-1.020e-01      2.740e+00    2785.6%
-3.536e-01      2.457e+00     795.0%
 8.492e-01      3.786e+00     345.8%
 1.152e+00      2.520e+00     118.7%
-1.868e-02     -1.282e+00    6765.1%
-1.307e-01      2.896e-01     321.6%
 4.855e-01      3.005e+00     518.9%
 8.988e-01     -3.620e+00     502.8%
-1.147e+00     -2.716e-01      76.3%
Exact        PM-sim
             (399741 shots)
 8.480e-01        3.845e-01    54.7%
-2.542e+00       -2.563e+00     0.8%
 7.712e-02       -1.727e-01   324.0%
-8.587e-01       -8.366e-01     2.6%
-1.164e+00       -1.470e+00    26.2%
-6.345e-01       -5.971e-01     5.9%
-1.015e+00       -1.207e+00    18.9%
-6.353e-01       -4.751e-01    25.2%
 1.555e+00        5.835e-01    62.5%
-1.332e+00       -1.504e+00    13.0%
Exact        PM-sim
             (208445 shots)
-6.979e-01       -1.463e+00   109.7%
-5.791e-02        3.672e-01   734.0%
 1.120e+00        1.573e+00    40.4%
-3.685e